# YANS 2025 Hackathon SFT

YANS 2025 で開催されるハッカソンの SFT サンプルコードです。  
ベースラインとなるデータセットの読み込み、学習、評価を行います。

## データセットの読み込み

ベースラインとなる学習データセット及び開発セットは Hugging Face のデータセット形式で提供しています。

https://huggingface.co/datasets/YANS-official/hackathon-2025-math

`datasets` ライブラリで読み込むことができます。

### 学習セット

500件です。
本ハッカソンの主眼はデータ作成です。
本データセットから派生させて学習データを作成しても構いませんし、本データセットを使用せずに新たにデータを作成しても構いません。

### 検証セット

400件です。
開発中のモデルの性能を測定することに使います。
開発中は、本ファイルに対する推論結果をリーダーボードに提出してください。

なお、モデルの最終評価は別のテストセットで行います。
基本的な問題の分布は検証セットと同様です。

In [1]:
from datasets import load_dataset

dataset = load_dataset("YANS-official/hackathon-2025-math")
print(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['question', 'answer_number'],
        num_rows: 500
    })
    dev: Dataset({
        features: ['question', 'answer_number'],
        num_rows: 400
    })
})


In [2]:
for i, item in enumerate(dataset["train"]):
    if i == 5:
        break
    print(item)

{'question': '魚を毎日2匹ずつ買っていたら、1週間で何匹になるでしょうか。', 'answer_number': 14}
{'question': '魚を毎日2匹ずつ購入しています。これを7日間継続した場合の累積数は？', 'answer_number': 14}
{'question': '0 + 0 はなんでしょう？', 'answer_number': 0}
{'question': 'たかしさんは自分の家族のためにスパゲッティとミートボールの食事を準備しています。たかしさんのミートボールのレシピでは、1個のミートボールにつき1/8ポンドのひき肉が必要です。たかしさんの家族は彼を含めて8人います。彼が4ポンドのひき肉を使ってミートボールを作り、家族のそれぞれが等しい数のミートボールを食べる場合、たかしさんは何個のミートボールを食べますか？', 'answer_number': 4}
{'question': '1冊12ページの本を3冊読みました。全部で何ページ読んだことになりますか？', 'answer_number': 36}


## SFT 学習

運営の提供する `train_sft` 関数を用いて学習を行ってください。

### レギュレーション
- 学習データのサンプル数は最大 500 件まで
- 学習データ内の文字数は最大50万文字まで
- 学習に用いるモデルは [SakanaAI/TinySwallow-1.5B](https://huggingface.co/SakanaAI/TinySwallow-1.5B) に限る

In [6]:
# ハッカソン用のコードをインストール
!pip install git+https://github.com/YANS-official/yans-2025-hackathon.git

  Cloning https://github.com/YANS-official/yans-2025-hackathon.git (to revision main) to /tmp/pip-req-build-gq473547
  Running command git clone --filter=blob:none --quiet https://github.com/YANS-official/yans-2025-hackathon.git /tmp/pip-req-build-gq473547
  Resolved https://github.com/YANS-official/yans-2025-hackathon.git to commit edd84e50931c7555cf53521f7bbc167acbe67c6c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for yans-2025-hackathon: filename=yans_2025_hackathon-0.1.0-py3-none-any.whl size=5243 sha256=50f1d97b189e93c64df4fb1905dfa994169bfa7f36262c3b58dcbb9eecb51e29
  Stored in directory: /tmp/pip-ephem-wheel-cache-3z_9a39f/wheels/f4/2e/87/39467f933b48b6daac0e0384b19bc652f8268bc9ed031044ea
Successfully built yans-2025-hackathon


バッチサイズや学習率といった学習時のハイパーパラメータは事前の実験で適切なものにセットしていますが、関数から指定できるものに限り、自由に変えていただいて構いません。

In [3]:
from yans_2025_hackathon import train_sft

help(train_sft)

Help on function train_sft in module yans_2025_hackathon.train_sft:

train_sft(train_dataset: datasets.arrow_dataset.Dataset, save_dir: str, model: str = 'SakanaAI/TinySwallow-1.5B', batch_size: int = 4, local_batch_size: int = 1, learning_rate: float = 2e-06, num_train_epochs: int = 3)
    YANS 2025 ハッカソンで使用する SFT（Supervised Fine-tuning）を実行する関数。
    学習データにはサイズ制限があり、最大500サンプル、総文字数50万文字まで。

    Args:
        train_dataset (Dataset): 学習用データセット。"messages"フィールドを含む必要がある。
        save_dir (str): 学習済みモデルの保存先ディレクトリ。
        model (str, optional): 使用する事前学習モデル。デフォルトは"SakanaAI/TinySwallow-1.5B"。
        batch_size (int, optional): バッチサイズ。デフォルトは4。
        local_batch_size (int, optional): デバイスごとのローカルバッチサイズ。デフォルトは1。
        learning_rate (float, optional): 学習率。デフォルトは2e-6。
        num_train_epochs (int, optional): 学習エポック数。デフォルトは3。

    Raises:
        ValueError: データセットに"messages"フィールドがない場合
        ValueError: サンプル数が500を超える場合
        ValueError: 総文字数が50万文字を超える場合



学習時のサンプルは以下の形式にフォーマットされている必要があります。

```python
[
  {"role": "user", "content": "1+1は？"},
  {"role": "assistant", "content": "2"},
]
```

ここでは例として、Hugging Face 形式のデータセットを上記の形に加工します。

In [22]:
from datasets import load_dataset


def transform_example(item: dict) -> dict:
    return {
        "messages": [
            {"role": "user", "content": item["question"]},
            {"role": "assistant", "content": str(item["answer_number"])},
        ]
    }


train_dataset = dataset["train"].map(
    transform_example, remove_columns=["question", "answer_number"]
)
train_dataset[0]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

{'messages': [{'content': '魚を毎日2匹ずつ買っていたら、1週間で何匹になるでしょうか。', 'role': 'user'},
  {'content': '14', 'role': 'assistant'}]}

In [23]:
from yans_2025_hackathon import train_sft

train_sft(
    train_dataset=train_dataset,
    save_dir="results",  # 学習後のモデルの保存場所
)

config.json:   0%|          | 0.00/709 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

Tokenizing train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,2.895400
20,1.395400
30,1.109400
40,1.041900
50,1.053700
60,0.902700
70,0.951300
80,0.934400
90,0.988300
100,0.991200


`save_dir` で指定したディレクトリにモデルが保存されています。

In [25]:
!ls results

added_tokens.json	model-00001-of-00002.safetensors  tokenizer.json
chat_template.jinja	model-00002-of-00002.safetensors  training_args.bin
config.json		model.safetensors.index.json	  vocab.json
generation_config.json	special_tokens_map.json
merges.txt		tokenizer_config.json


## モデルの評価

学習したモデルを開発セットで検証してみましょう。
推論には [vLLM](https://github.com/vllm-project/vllm) ライブラリを使用します。
Hugging Face の推論そのままに比べて高速に応答を生成できます。

In [26]:
!pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.4/414.4 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.0/821.0 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/

In [6]:
from vllm import LLM

# 学習したモデルのパスを指定
model = LLM("./results")

INFO 09-05 09:59:32 [__init__.py:241] Automatically detected platform cuda.
INFO 09-05 09:59:33 [utils.py:326] non-default args: {'model': './results', 'disable_log_stats': True}
INFO 09-05 09:59:47 [__init__.py:711] Resolved architecture: Qwen2ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 09-05 09:59:47 [__init__.py:2816] Downcasting torch.float32 to torch.bfloat16.
INFO 09-05 09:59:47 [__init__.py:1750] Using max model len 32768
INFO 09-05 09:59:50 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 09-05 09:59:50 [__init__.py:2921] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 09-05 10:00:31 [llm.py:298] Supported_tasks: ['generate']


In [10]:
batch_responses = model.chat([{"role": "user", "content": "1 + 1 はなんですか？"}])
print(batch_responses[0].outputs[0].text)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

2


In [11]:
from datasets import load_dataset

dataset = load_dataset("YANS-official/hackathon-2025-math")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer_number'],
        num_rows: 500
    })
    dev: Dataset({
        features: ['question', 'answer_number'],
        num_rows: 400
    })
})


In [21]:
from vllm import SamplingParams

dev_inputs = [
    [{"role": "user", "content": item["question"]}] for item in dataset["dev"]
]
dev_batch_responses = model.chat(
    dev_inputs,
    SamplingParams(temperature=0.0),  # 貪欲デコーディング
)

Adding requests:   0%|          | 0/400 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/400 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [22]:
from yans_2025_hackathon import evaluate_response

num_total = 0
num_correct = 0
for resopnse, item in zip(dev_batch_responses, dataset["dev"]):
    response_text = resopnse.outputs[0].text
    answer_number = item["answer_number"]
    is_correct = evaluate_response(response_text, answer_number)

    num_total += 1
    num_correct += int(is_correct)
print("正解率", num_correct / num_total)

正解率 0.0825


## リーダーボードへの提出

リーダーボードには、以下のように元の質問（`"question"`）と、モデルの応答（`"response"`）を含む JSONL ファイルを提出してください。

```jsonl
{"question": "...", "response": "..."}
{"question": "...", "response": "..."}
```

`question` の値を用いて、適切な答えとの照合を行いますので、順番は気にしなくても構いません。

In [23]:
import json

# vLLM の出力と、開発データセットの質問を整形し、JSONL ファイルとして書き出す
with open("predictions.jsonl", "w") as f:
    for response, item in zip(dev_batch_responses, dataset["dev"]):
        f.write(
            json.dumps(
                {"question": item["question"], "response": response.outputs[0].text},
                ensure_ascii=False,
            )
            + "\n"
        )

作成したファイルは、ノートブック左の 📁 アイコンをクリックし、そこに表示されるファイル一覧からダウンロードすることができます（[参考](https://www.kikagaku.co.jp/kikagaku-blog/google-colab-file/#i-3)）。

## OpenAI API の使い方

データを加工したり、作成するために OpenAI API を使うことができます。
以下のコードを参考にしてください。

In [24]:
!pip install openai

In [25]:
import os
from openai import OpenAI

client = OpenAI(
    api_key="sk-...",  # 配布された API キーを使用する
)

response = client.responses.create(
    model="gpt-4o-mini",  # 使用可能モデルは gpt-4o-mini のみ
    instructions="数学の問題を解いてください。応答の末尾に必ず「答え:」という形式で解答となる数字を出力してください。",
    input="たけしさんは飴を４個もらいましたが、２個食べました。残りの飴の数は何個？",
)

print(response.output_text)

たけしさんがもらった飴は4個で、食べたのが2個です。したがって、残りの飴の数は次のように計算します。

4個 - 2個 = 2個

答え: 2
